### Sample script for item-based collaborative filtering  
#### Execute recommendation  

#### Import libraries

In [ ]:
import numpy as np
import pandas as pd

#### Parameters  

In [ ]:
csv_in = 'sushi_corr-min4.csv'
# min number of common items between target user's evaluation and items in DB
min_common_items = 4

# To show more rows and columns
pd.options.display.max_rows = 999 
pd.options.display.max_columns = 999 

#### Read CSV file  

In [ ]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
df.index = df.columns
print(df.shape)
print(df.info())
display(df.head())

In [ ]:
def predict_scores(df_sim, ser_target):
    ret = {}
    for item1 in df_sim.index:  # not yet rated by the target user
        v1 = df_sim.loc[item1]
        #if v1.isnull().sum() > 0:  # debug
        #    print('v1:',v1)  # debug
        if v1.notnull().sum() < min_common_items: continue
        v11 = v1[ v1.notnull() ]
        t11 = ser_target[ v1.notnull() ]
        pred1 = (v11 * t11).sum() / np.abs(v11).sum()
        #print('v11:',v11)  # debug
        #print('t11:',t11)  # debug
        #print('pred1:',pred1)  # debug
        ret[item1] = pred1
    
    ser_ret = pd.Series(ret)
    
    return ser_ret.sort_values(ascending=False)

Function for user-based collaborative filtering.  

arguments: dictionary of scores for the target user  
and the number of items to recommend.  

ex)
```
get_recomm_by_user_sim(df, {'maguro':1, 'ika':1, 'uni':3,
                        'awabi':4, 'hirame':4, 'aoyagi':4})  
```
-> return list such as [('akagai', 2.9835603009918303), ('mirugai', 2.945676429588114), ...]

In [ ]:
def get_recomm_by_item_sim(df, target_dic):
    ser_target = pd.Series(target_dic)
    #print(target_dic)  # debug
    #print(ser_target)  # debug
    # make dataframe with columns included in target_dic
    #print(df.shape)  # debug
    df_scores = df[ ser_target.index ]
    #print(df_scores.shape)  # debug
    # drop rows included in target_dic (already rated)
    df_scores = df_scores.drop(index=ser_target.index)
    #print(df_scores.shape)  # debug
    #display(df_scores.head())  # debug
    recomm = predict_scores(df_scores, ser_target)
    
    return recomm

#### Do recommendation  

In [ ]:
recomm = get_recomm_by_item_sim(df,
                                {'maguro':1, 'ika':1, 'uni':3,
                                 'awabi':4, 'hirame':4, 'aoyagi':4})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

recomm = get_recomm_by_item_sim(df,
                                {'anago': 3, 'maguro': 4, 'ikura': 3,
                                 'hamachi': 4, 'samon': 4, 'unagi': 3,
                                 'suzuki': 2, 'hamo': 1, 'nasu': 1,
                                 'shiso_maki': 2})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())